In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By

#HEADERS ={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

In [4]:
# https://www.pycodemates.com/2022/01/Indeed-jobs-scraping-with-python-bs4-selenium-and-pandas.html

In [14]:
options = Options()
options.add_argument("--headless")
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
# driver = webdriver.Firefox(options=options)
# driver.get('http://google.com/')

In [15]:
def get_test(url, job_title, location):
    driver = webdriver.Firefox(options=options)
    driver.get(url)
    time.sleep(3)
    driver.find_element(By.XPATH,'//*[@id="text-input-what"]').send_keys(job_title)
    time.sleep(3)
    locations = driver.find_element(By.XPATH,'//*[@id="text-input-where"]')
    if locations.get_attribute("value") == "":
        driver.find_element(By.XPATH,'//*[@id="text-input-where"]').send_keys(location)
    time.sleep(3)
    driver.find_element(By.XPATH,'/html/body/div').click()
    time.sleep(3)
    try:
        driver.find_element(By.XPATH,'//*[@id="jobsearch"]/button').click()
    except:
        driver.find_element(By.XPATH,'//*[@id="whatWhereFormId"]/div[3]/button').click()
    current_url = driver.current_url
    return current_url
    
current_url = get_test('https://indeed.com/','Data Scientist',"Los Angeles")

In [16]:
def scrape_job_details(url):
    
    resp = requests.get(url)
    content = BeautifulSoup(resp.content, 'lxml')
   
    jobs_list = []    
    for post in content.select('.job_seen_beacon'):
        try:
            data = {
                "job_title":post.select('.jobTitle')[0].get_text().strip(),
                "company":post.select('.companyName')[0].get_text().strip(),
                "rating":post.select('.ratingNumber')[0].get_text().strip(),
                "location":post.select('.companyLocation')[0].get_text().strip(),
                "date":post.select('.date')[0].get_text().strip(),
                "job_desc":post.select('.job-snippet')[0].get_text().strip(),
                "job_link":post.find(class_='jcs-JobTitle', href=True)['href']
            }
        except IndexError:
            continue          
        jobs_list.append(data)

    dataframe = pd.DataFrame(jobs_list)
    dataframe["job_link"] = 'https://www.indeed.com' + dataframe['job_link'].astype(str)
     
    return dataframe

lol = scrape_job_details(current_url)
# rename columns
lol["date"] = [x.replace('Posted','') for x in lol["date"]]
lol["job_title"] = [x.replace('new', '') for x in lol["job_title"]]

In [17]:
lol

,job_title,company,rating,location,date,job_desc,job_link
0,Ecommerce Data Scientist - Remote Position,Stingray Direct,3.7,"Remote in Woodland Hills, CA 91367",Just posted,The Ecommerce Data Scientist must have experie...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,Data Scientist,UCLA Health,4.0,"Los Angeles, CA 90095+1 location",7 days ago,This position will support the organization’s ...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
2,DATA SCIENTIST,Los Angeles County Department of Human Resources,4.1,"Los Angeles County, CA+3 locations",22 days ago,Four (4) years of experience applying machine ...,https://www.indeed.com/rc/clk?jk=6dd8d2a7c3fe7...
3,Data Scientist - Recommendations,Live Nation,4.1,"Remote in West Hollywood, CA 90069+1 location",30+ days ago,This person will support and improve our recom...,https://www.indeed.com/rc/clk?jk=ba6f99e6f2792...
4,Data Scientist - Analytics,Twitch,4.4,"Remote in Los Angeles, CA",30+ days ago,Develop scalable data pipelines to support ana...,https://www.indeed.com/rc/clk?jk=6c15755a1bb6e...
5,Data Scientist,Public Storage,3.3,"Glendale, CA",17 days ago,Strong command of applied math (linear/matrix ...,https://www.indeed.com/rc/clk?jk=5fda368a567f6...
6,Data Scientist II,JPL/NASA,4.3,"Pasadena, CA 91109",1 day ago,Support the development of software tools for ...,https://www.indeed.com/rc/clk?jk=5ea20048fc96a...
7,SENIOR DATA SCIENTIST,Los Angeles County Department of Human Resources,4.1,"Los Angeles County, CA+1 location",22 days ago,Positions allocable to this class typically re...,https://www.indeed.com/rc/clk?jk=e964a6095745c...
8,Data Scientist/Senior Data Scientist,UCLA,4.2,"Remote in Los Angeles, CA",2 days ago,Partner with nontechnical stakeholders to set ...,https://www.indeed.com/rc/clk?jk=7f530373e5253...
9,Data Scientist,Circle,3.7,"Remote in Los Angeles, CA",30+ days ago,Perform deep analyses and build models to unde...,https://www.indeed.com/rc/clk?jk=8e64304bbe31e...


In [7]:
lol.iloc[6]["job_link"]

'https://www.indeed.com/rc/clk?jk=9d2e4e01c26874be&fccid=6824fc1f087bd63e&vjs=3'

In [62]:
def scrape_job_descr():
    job_link_list = lol["job_link"].tolist()
    
    for url in job_link_list:
        #get response from URL
        resp = requests.get(url)
        
        #create soup object
        soup = BeautifulSoup(resp.content, 'lxml')
        job_desc_soup = []
        job_desc_soup += soup.select('div#jobDescriptionText.jobsearch-jobDescriptionText')
#         for tag in job_desc_soup:
#             test = tag.get('p')
#             print(test)
        
        # incomplete
        # still need to use string manip to remove <p><b></b></p>
        # still need to deal with non <p> elements
#         job_desc_soup = soup.select("p.jobSearch-jobDescriptionText")
# #         find_all('p')
#         for item in job_desc_soup:
#             print(item.get())
        
#         .getText()
        
#         job_desc_soup = soup.select('.jobSearch-jobDescriptionText')
#         for tag in job_desc_soup:
#             test = tag.get('p')
        
    return job_desc_soup

scrape_job_descr()

[<div class="jobsearch-jobDescriptionText" id="jobDescriptionText">TikTok is the leading destination for short-form mobile video. Our mission is to inspire creativity and bring joy. 
 <br/>
 <br/> Building a world where people can safely discover, create and connect. 
 <br/>
 <br/> The Trust &amp; Safety team at TikTok helps ensure that our global community is safe and empowered to create and enjoy content across all of our applications. 
 <br/>
 <br/> We are seeking an experienced Lead Threat Data Scientist with an investigative and adversarial mindset to help us identify our gaps before nefarious actors beat us to it. You will be using your subject matter expertise in technical feature abuse to data mine threat actor abuse tactics, develop predictive models of feature abuse, and analyze proposed features to identify existing or projected exploits of product features by threat actors. The Lead Threat Data Scientist will build out a vertical of the Product Exploitation Hunt Team, condu